In [1]:
import keras

Using TensorFlow backend.


In [3]:
from keras import layers
import numpy as np

latent_dim=32
h=32
w=32
c=3
generator_input=keras.Input(shape=(latent_dim,))



In [10]:
x=layers.Dense(128*16*16)(generator_input)
x=layers.LeakyReLU()(x)
x=layers.Reshape((16,16,128))(x)
x=layers.Conv2D(256,5,padding='same')(x)



In [11]:
x=layers.LeakyReLU()(x)
x=layers.Conv2DTranspose(256,4,strides=2,padding='same')(x)

x=layers.LeakyReLU()(x)
x=layers.Conv2D(256,5,padding='same')(x)

x=layers.LeakyReLU()(x)
x=layers.Conv2D(256,5,padding='same')(x)

x=layers.LeakyReLU()(x)

x=layers.Conv2D(c,7,activation='tanh',padding='same')(x)


In [13]:

generator=keras.models.Model(generator_input,x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [14]:
discriminator_input=layers.Input(shape=(h,w,c))


In [16]:
x=layers.Conv2D(128,3)(discriminator_input)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128,4,strides=2)(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2D(128,4,strides=2)(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2D(128,4,strides=2)(x)
x=layers.LeakyReLU()(x)
x=layers.Flatten()(x)




x=layers.Dropout(0.4)(x)
x=layers.Dense(1,activation='sigmoid')(x)


from keras.models import Model
discriminator=Model(discriminator_input,x)

discriminator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [19]:
discriminator_optimizer=keras.optimizers.RMSprop(lr=0.0008,clipvalue=1.0,decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')

In [20]:
discriminator.trainable=False

gan_input=keras.Input(shape=(latent_dim,))
gan_output=discriminator(generator(gan_input))
gan=Model(gan_input,gan_output)
from keras.optimizers import RMSprop
gan_optimizer=RMSprop(lr=0.0004,clipvalue=1.0,decay=1e-8)

gan.compile(optimizer=gan_optimizer,loss='binary_crossentropy')


Now we can start training. To recapitulate, this is schematically what the training loop looks like:

for each epoch:
    * Draw random points in the latent space (random noise).
    * Generate images with `generator` using this random noise.
    * Mix the generated images with real ones.
    * Train `discriminator` using these mixed images, with corresponding targets, either "real" (for the real images) or "fake" (for the generated images).
    * Draw new random points in the latent space.
    * Train `gan` using these random vectors, with targets that all say "these are real images". This will update the weights of the generator (only, since discriminator is frozen inside `gan`) to move them towards getting the discriminator to predict "these are real images" for generated images, i.e. this trains the generator to fool the discriminator.
Let's implement it:

In [28]:
import os
from keras.preprocessing import image
(x_train,y_train),(_,_)=keras.datasets.cifar10.load_data()

x_train=x_train[y_train.flatten()==6]

x_train=x_train.reshape(
    
    (x_train.shape[0],)+(h,w,c)).astype('float32')/255.

iter=10000
batch_size=20
save_dir='./gan_images/'
os.mkdir(save_dir)
start=0
for step in range(iter):
    random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
    
    generated_images=generator.predict(random_latent_vectors)
    
    stop=start+batch_size
    real_images=x_train[start:stop]
    combined_images=np.concatenate([generated_images,real_images])
    
    labels=np.concatenate([np.ones((batch_size,1)),
                          np.zeros((batch_size,1))])
    
    labels+=0.05*np.random.random(labels.shape)
    d_loss=discriminator.train_on_batch(combined_images,labels)
    
    random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
    misleading_targets=np.zeros((batch_size,1))
    
    a_loss=gan.train_on_batch(random_latent_vectors,misleading_targets)
    
    start+=batch_size
    
    if start>len(x_train)-batch_size:
        start=0
        
    if step%100==0:
        gan.save_weights('gan.h5')
        
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        img=image.array_to_img(generated_images[0]*255,scale=False)
        img.save(os.path.join(save_dir,'real_frog'+str(step)+'.png'))
        
        

D:\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss at step 0: 2.90828
adversarial loss at step 0: 15.810432
discriminator loss at step 100: 0.63443935
adversarial loss at step 100: 1.0343671
discriminator loss at step 200: 0.71812767
adversarial loss at step 200: 0.69485116
discriminator loss at step 300: 0.71559393
adversarial loss at step 300: 0.7613785
discriminator loss at step 400: 0.69547176
adversarial loss at step 400: 0.7444247
discriminator loss at step 500: 0.6910823
adversarial loss at step 500: 0.7347973
discriminator loss at step 600: 0.70495445
adversarial loss at step 600: 0.7769234
discriminator loss at step 700: 0.68687665
adversarial loss at step 700: 0.75351006
discriminator loss at step 800: 0.7004467
adversarial loss at step 800: 0.6925286
discriminator loss at step 900: 0.7083837
adversarial loss at step 900: 0.7518848
discriminator loss at step 1000: 0.691304
adversarial loss at step 1000: 0.80026805
discriminator loss at step 1100: 0.6881939
adversarial loss at step 1100: 0.7618004
discrimina

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Sample random points in the latent space
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# Decode them to fake images
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()